In [5]:
import os
import json 
import requests
import openai
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv  
import os  
import openai
import os  
  

# Configure OpenAI API

AZURE_OPENAI_API_VERSION = "2023-03-15-preview"
AZURE_OPENAI_ENDPOINT ="https://cog-42as6n6i4ldsi.openai.azure.com/"
AZURE_OPENAI_API_KEY ="653103af9cd04492a2232b70327b7ce0"

# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"

COGNITIVE_SEARCH_INDEX = "finance-complaint-data"
COGNITIVE_SEARCH_HOSTNAME = "https://gptkb-42as6n6i4ldsi.search.windows.net"
COGNITIVE_SEARCH_API_KEY = "kSovyGNsoVRCGtovPviVe9Kibz7351x2Wb7RCHEjBRAzSeDWVLvj"
COGNITIVE_SEARCH_API_VERSION = "2023-07-01-preview"

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
#
import json
import math
import pandas as pd
import requests
from azure.storage.blob import BlobServiceClient

# Define the connection string and container name
conn_str = "DefaultEndpointsProtocol=https;AccountName=entchatgptstr;AccountKey=gZss7VuVlbiwShlfML34GaeAw3QYEyicbZLF9D47zkGa/GCgx6jNGcNGBz0qqRa4Ci3MPqXrWI2O+AStm9gdMg==;EndpointSuffix=core.windows.net"
container_name = "dftrain100"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get a reference to the blob container
container_client = blob_service_client.get_container_client(container_name)

# Define the maximum document size in bytes
MAX_DOC_SIZE = 12 * 1024 * 1024  # 16 MB

# Get a reference to the blob
blob_client = container_client.get_blob_client("df_test.json")

# Download the blob as a string
data_str = blob_client.download_blob().content_as_text()

# Load the string into a Python object
data_list = json.loads(data_str)

# Split the data into chunks of maximum size MAX_DOC_SIZE
data_chunks = [data_list[i:i+MAX_DOC_SIZE] for i in range(0, len(data_list), MAX_DOC_SIZE)]

# Upload each chunk as a separate document
for i, chunk in enumerate(data_chunks):
    # Convert the Python object to a Pandas DataFrame
    data_df = pd.DataFrame(chunk)

    # Convert float values to string representation
    for col in data_df.columns:
        if data_df[col].dtype == "float64":
            data_df[col] = data_df[col].apply(lambda x: str(x) if not math.isnan(x) else None)

    # Convert the Pandas DataFrame to a list of dictionaries
    data_dict = data_df.to_dict(orient="records")

    # Create the upload data payload
    upload_data = {
        "value": [
            {
                "@search.action": "upload",
                **record,
            } for record in data_dict
        ]
    }

    # Set the headers for the request
    headers = {
        "Content-Type": "application/json",
        "api-key": COGNITIVE_SEARCH_API_KEY
    }

    # Send the POST request to the search index API
    url = f"{COGNITIVE_SEARCH_HOSTNAME}/indexes/{COGNITIVE_SEARCH_INDEX}/docs/index?api-version={COGNITIVE_SEARCH_API_VERSION}"
    response = requests.post(url, headers=headers, json=upload_data)

    # Print the response status code and message
    print(f"Uploaded chunk {i+1}/{len(data_chunks)} - Status {response.status_code}, message: {response.text}")


Uploaded chunk 1/1 - Status 413, message: The page was not displayed because the request entity is too large.


In [3]:


#data_json = json.load(open("sample_data.json"))

headers = {
    "Content-Type": "application/json",
    "api-key": COGNITIVE_SEARCH_API_KEY
}

upload_data = {
    "value": [
        {
            "@search.action": "upload",
            **df_test,
            #"issue_statement_vector": embeddings.embed_query(data_json['issue_statement']),
           # "comments_vector": embeddings.embed_query(data_json['comments']),
            #"closure_comments_vector": embeddings.embed_query(data_json['closure_comments'])
        }
    ]
}

print(upload_data)

url = f"{COGNITIVE_SEARCH_HOSTNAME}/indexes/{COGNITIVE_SEARCH_INDEX}/docs/index?api-version={COGNITIVE_SEARCH_API_VERSION}"
response = requests.post(url, headers=headers, json=upload_data)
print(f"Uploaded data - Status {response.status_code}, message: {response.text}")

{'value': [{'@search.action': 'upload', 'Date received': 0       2021-09-17
1       2020-05-26
2       2022-05-01
3       2019-10-04
4       2022-01-09
           ...    
9995    2022-11-11
9996    2023-05-04
9997    2021-08-26
9998    2019-03-07
9999    2022-11-23
Name: Date received, Length: 10000, dtype: object, 'Product': 0       Credit reporting, credit repair services, or o...
1                                         Debt collection
2       Credit reporting, credit repair services, or o...
3       Money transfer, virtual currency, or money ser...
4       Credit reporting, credit repair services, or o...
                              ...                        
9995    Credit reporting, credit repair services, or o...
9996                                      Debt collection
9997    Credit reporting, credit repair services, or o...
9998    Credit reporting, credit repair services, or o...
9999    Credit reporting, credit repair services, or o...
Name: Product, Length: 10000, dtyp

TypeError: Object of type Series is not JSON serializable